In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, pandas_udf
from pyspark.sql.types import StringType, StructType

import cv2
from ultralytics import YOLO
import numpy as np
import base64
import pandas as pd
import json
import os

CAMERA_RAW_TOPIC = "camera_raw"
CAMERA_OUT_TOPIC = "camera_track"
BOOTSTRAP_SERVER = "localhost:9092"
JPEG_QUALITY = 80

model = YOLO("../models/yolo11n.pt")

# os.environ['HADOOP_HOME'] = 'C:/hadoop'
# os.environ['PYSPARK_PYTHON'] = 'python'
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'python'

### Khai báo ứng dụng Spark ###
spark = (
    SparkSession.builder
    .appName("spark_video_processing")
    .master("local")
    .config("spark.driver.memory", "2g")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    .config("spark.sql.execution.arrow.maxRecordsPerBatch", "40")
    .config("spark.streaming.kafka.consumer.poll.ms", "500")
    .config("spark.kafka.consumer.connection.timeout.ms", "10000")
    .config("spark.kafka.consumer.request.timeout.ms", "305000")
    .config("spark.rpc.message.maxSize", "2047")
    .config("spark.ui.port", "8041")
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.4")

    # .config("spark.hadoop.io.native.lib.available", "false")
    # .config("spark.executor.extraJavaOptions", "-Djava.io.tmpdir=C:/tmp")
    # .config("spark.driver.extraJavaOptions", "-Djava.io.tmpdir=C:/tmp")

    .getOrCreate()
)

### Khai báo schema dữ liệu được gửi đến ###

schema = (
    StructType()
    .add("camera_id", StringType())
    .add("frame", StringType())
    .add("timestamp", StringType())
)

### Khai báo dataframe đọc luồng dữ liệu ###
df = (
    spark.readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", BOOTSTRAP_SERVER)
    .option("subscribe", CAMERA_RAW_TOPIC)
    .load()
)

df = df.selectExpr("CAST(value AS STRING)")
df = df.select(from_json(col("value"), schema).alias("data")).select("data.*")


@pandas_udf("string")
def track_objects_udf(pdf: pd.DataFrame) -> pd.Series:
    """
    Detect tất cả box nhét vào results, 
    rồi trả về dưới dạng pandas.Series
    """

    results = []

    for row in pdf.itertuples():
        img_bytes = base64.b64decode(row.frame)
        np_img = np.frombuffer(img_bytes, np.uint8)
        frame = cv2.imdecode(np_img, cv2.IMREAD_COLOR)

        tracks = model.track(frame, persist=True)
        res = tracks[0].boxes.xyxy.tolist() if tracks else []
        payload = json.dumps({
            "camera_id": row.camera_id,
            "timestamp": row.timestamp,
            "raw_frame": row.frame,
            "detections": res
        })
        results.append(payload)


    return pd.Series(results)


processed_df = df.withColumn("result", track_objects_udf(df))

processed_df.selectExpr("CAST(result AS STRING) AS value") \
    .writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", BOOTSTRAP_SERVER) \
    .option("topic", CAMERA_OUT_TOPIC) \
    .start() \
    .awaitTermination()

Py4JJavaError: An error occurred while calling o53.load.
: java.lang.NoSuchMethodError: 'scala.collection.mutable.WrappedArray scala.Predef$.wrapRefArray(java.lang.Object[])'
	at org.apache.spark.sql.kafka010.KafkaSourceProvider$.<init>(KafkaSourceProvider.scala:545)
	at org.apache.spark.sql.kafka010.KafkaSourceProvider$.<clinit>(KafkaSourceProvider.scala)
	at org.apache.spark.sql.kafka010.KafkaSourceProvider.org$apache$spark$sql$kafka010$KafkaSourceProvider$$validateStreamOptions(KafkaSourceProvider.scala:338)
	at org.apache.spark.sql.kafka010.KafkaSourceProvider.sourceSchema(KafkaSourceProvider.scala:71)
	at org.apache.spark.sql.execution.datasources.DataSource.sourceSchema(DataSource.scala:244)
	at org.apache.spark.sql.execution.datasources.DataSource.sourceInfo$lzycompute(DataSource.scala:129)
	at org.apache.spark.sql.execution.datasources.DataSource.sourceInfo(DataSource.scala:129)
	at org.apache.spark.sql.execution.streaming.StreamingRelation$.apply(StreamingRelation.scal